<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-&amp;-Inits" data-toc-modified-id="Imports-&amp;-Inits-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports &amp; Inits</a></span></li><li><span><a href="#Data-Preparation" data-toc-modified-id="Data-Preparation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Preparation</a></span></li></ul></div>

# AG News Classifier with ConvNet
Classifier to classify news titles into categories.

## Imports & Inits

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pdb
import pandas as pd
import numpy as np
import torch

from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from pathlib import Path

from ignite.engine import Events, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.contrib.handlers import ProgressBar

In [4]:
from consts import consts
from ag.data import NewsDataset, DataContainer

consts

Namespace(batch_size=64, checkpointer_name='classifier', checkpointer_prefix='cbow', device='cuda:3', early_stopping_criteria=5, embedding_size=100, learning_rate=0.0001, metric_file=PosixPath('../data/ag_news/work_dir/metrics.csv'), model_dir=PosixPath('../data/ag_news/work_dir/models'), num_epochs=100, path=PosixPath('../data/ag_news'), proc_dataset_csv=PosixPath('../data/ag_news/news_with_splits.csv'), raw_dataset_csv=PosixPath('../data/ag_news/news.csv'), save_every=2, save_total=5, vectorizer_file=PosixPath('../data/ag_news/work_dir/vectorizer.json'), work_dir=PosixPath('../data/ag_news/work_dir'))

## Data Preparation

In [5]:
df = pd.read_csv(consts.proc_dataset_csv)
df.head()

,category,split,title
0,Business,train,"Jobs, tax cuts key issues for Bush"
1,Business,train,Jarden Buying Mr. Coffee #39;s Maker
2,Business,train,Retail sales show festive fervour
3,Business,train,Intervoice's Customers Come Calling
4,Business,train,Boeing Expects Air Force Contract


In [8]:
dc = DataContainer(df, NewsDataset, consts.vectorizer_file, consts.batch_size)

In [12]:
itr = iter(dc.train_dl)

In [14]:
x,y = next(itr)